## 売上資料Ⅲ
## 月別Class別値引表　作成

- 作成年度と作成月の入力を忘れずに！

In [ ]:
import numpy as np
import pandas as pd
import pickle

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def data_read():
    # 売上
    target_col = ['年度','売上月','請求先CD','伝票区分','大分類','品目','品名','規格','販売金額']
    uriagedf_i = pd.read_csv("/content/drive/MyDrive/Purchased_products/csv/I_U_2009_2022.csv",usecols=target_col)
    uriagedf_i = uriagedf_i[uriagedf_i['請求先CD'] != 80004]
    uriagedf_i = uriagedf_i[uriagedf_i['請求先CD'] != 0]

    # Class、大分類各dict
    with open('/content/drive/MyDrive/Purchased_products/class_dict.pkl', 'rb') as f:
        class_dict = pickle.load(f)
    with open('/content/drive/MyDrive/Purchased_products/dai_dict.pkl', 'rb') as f:
        dai_dict = pickle.load(f)

    return uriagedf_i, class_dict, dai_dict

In [ ]:
# 作成年度を入力
nendo = 2021
# 作成月を入力（1～12を入力）
month = 5

# Excel上に得意先コードの有無をcheck　OKなら以下を進める
# データ読込
uriagedf_i, class_dict, dai_dict = data_read()

In [ ]:
def month_set(nendo, month):
    """年と月で6桁の年月にする
    """
    if month < 6:
        this_month = int(str(nendo+1) + "0" + str(month))
    else:
        if month > 9:
            this_month = int(str(nendo) + str(month))
        else:
            this_month = int(str(nendo) + "0" + str(month))
    
    return this_month

In [ ]:
# 当月を設定
this_month = month_set(nendo, month)

# 当年の最終月を設定
end_month = month_set(nendo, 5)

# 12ヵ月分のlistを作成
list_12 = []
m = 6
y = month_set(nendo, m)
list_12.append(y)
for i in range(11):
    m += 1
    if m > 12:
        m = 1
    y = month_set(nendo, m)
    list_12.append(y)
print('当月：', this_month)
print('最終月', end_month)
print(list_12)

当月： 202205
最終月 202205
[202106, 202107, 202108, 202109, 202110, 202111, 202112, 202201, 202202, 202203, 202204, 202205]


In [ ]:
# uriagedf_i 整理
uriagedf_i = uriagedf_i.fillna(0)
uriagedf_i["請求先CD"] = uriagedf_i["請求先CD"].astype(int)
uriagedf_i["売上月"] = uriagedf_i["売上月"].astype(int)
uriagedf_i["販売金額"] = uriagedf_i["販売金額"].astype(int)
uriagedf_i["年度"] = uriagedf_i["年度"].astype(int)

### データ作成

In [ ]:
# 売上データ絞り込み
uriagedf_i = uriagedf_i[(uriagedf_i["売上月"] >=list_12[0]) & (uriagedf_i["売上月"] <= list_12[-1])]
uriagedf_i = uriagedf_i[uriagedf_i["伝票区分"] == "値引"]

# Class、大分類をセット
uriagedf_i['大分類CD'] = uriagedf_i['大分類'].str[:4]

class_list = []
for i in range(len(uriagedf_i)):
    class_list.append(dai_dict[uriagedf_i.iloc[i]['大分類CD']][1])
uriagedf_i['Classcd'] = class_list

In [ ]:
# Class別月別データセット
nebiki_df = uriagedf_i.groupby(['Classcd','売上月'],as_index=False).sum()
nebiki_df = nebiki_df.sort_values(['Classcd','売上月'])

nebiki_list = []
for i in range(len(nebiki_df)):
    tmp_list = []
    tmp_list.append(nebiki_df.iloc[i]["Classcd"])
    tmp_list.append(class_dict[nebiki_df.iloc[i]["Classcd"]])
    tmp_list.append(nebiki_df.iloc[i]['売上月'])
    tmp_list.append(list_12.index(nebiki_df.iloc[i]['売上月']))
    tmp_list.append(nebiki_df.iloc[i]["販売金額"])
    nebiki_list.append(tmp_list)

In [ ]:
col_nm = ['Classcd','Class名','売上月','月番号','金額']

In [ ]:
nebiki_df = pd.DataFrame(nebiki_list, columns=col_nm)

In [ ]:
nebiki_df

In [ ]:
# Excelデータ作成
nebiki_df.to_excel("/content/drive/MyDrive/Purchased_products/excel/月別Class別値引表.xlsx", sheet_name='値引')